In [2]:
PATIENTID = '751E1F62-93C3-8048-7F77-48415A978963'
PATIENTID = PATIENTID.upper()
print(PATIENTID)

751E1F62-93C3-8048-7F77-48415A978963


In [3]:
neo4j_url = "bolt://localhost:7687" 
neo4j_user = "neo4j" 
neo4j_password = "Apurv1234567890"
neo4j_db = "poc"
SOURCE_DATASET_LOCATION = "/Users/amk/#Datasets/synthea/csv/synthea50"

In [4]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
        .master('spark://Adityas-MacBook-Pro.local:7077')
        .appName('HealthConnect Test')
        .config('spark.ui.port', '4051')
        .config('spark.sql.repl.eagerEval.enabled', True)
        .config('spark.jars', '/Users/amk/Github/healthconnect/damg7374/code/utilities/neo4j-spark-connector.jar')
        .config("neo4j.url", neo4j_url)
        .config("neo4j.authentication.type", "basic")
        .config("neo4j.authentication.basic.username", neo4j_user)
        .config("neo4j.authentication.basic.password", neo4j_password)
        .config("neo4j.authentication.basic.password", neo4j_password)
        .config("neo4j.database", neo4j_db)
        .config('spark.executor.instances', 1)
        .config('spark.executor.cores', 1) 
        .config('spark.executor.memory', '1G') 
        .config('spark.cores.max', 1) 
        .getOrCreate())


24/12/05 01:20:18 WARN Utils: Your hostname, Adityas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.82 instead (on interface en0)
24/12/05 01:20:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/05 01:20:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
print("================")
print("patients.csv")
print("================")
SOURCE_DATASET_FILENAME = "patients.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(f'upper(Id) = "{PATIENTID}"')
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")


patients.csv


24/12/05 01:20:29 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------------------------+----------+---------+-----------+---------+----------+------+-------+----------+----+------+-------+-------+-----+-----------+------+--------------------------+----------------------------+--------+-------------+--------------+-----+----+-----------------+------------------+-------------------+-------------------+------+
|Id                                  |BIRTHDATE |DEATHDATE|SSN        |DRIVERS  |PASSPORT  |PREFIX|FIRST  |MIDDLE    |LAST|SUFFIX|MAIDEN |MARITAL|RACE |ETHNICITY  |GENDER|BIRTHPLACE                |ADDRESS                     |CITY    |STATE        |COUNTY        |FIPS |ZIP |LAT              |LON               |HEALTHCARE_EXPENSES|HEALTHCARE_COVERAGE|INCOME|
+------------------------------------+----------+---------+-----------+---------+----------+------+-------+----------+----+------+-------+-------+-----+-----------+------+--------------------------+----------------------------+--------+-------------+--------------+-----+----+--

In [6]:
print("================")
print("encounters.csv")
print("================")

SOURCE_DATASET_FILENAME = "encounters.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(f'upper(PATIENT) = "{PATIENTID}"').orderBy('Id')

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

encounters.csv
+------------------------------------+-------------------+-------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+--------------+---------+----------------------------------------------------------------+-------------------+----------------+--------------+----------+-----------------------------------+
|Id                                  |START              |STOP               |PATIENT                             |ORGANIZATION                        |PROVIDER                            |PAYER                               |ENCOUNTERCLASS|CODE     |DESCRIPTION                                                     |BASE_ENCOUNTER_COST|TOTAL_CLAIM_COST|PAYER_COVERAGE|REASONCODE|REASONDESCRIPTION                  |
+------------------------------------+-------------------+-------------------+------------------------------------+------------------------------------+-

In [7]:
# Select the Encounter ID
ENCOUNTERID = 'B41D26FD-733E-7773-0115-A9E5C0451FBC'
ENCOUNTERID = ENCOUNTERID.upper()
print(ENCOUNTERID)

a = df.filter(f'upper(Id) = "{ENCOUNTERID}"')

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

a = a = df.filter(f'upper(Id) = "{ENCOUNTERID}"').orderBy('Id').selectExpr('Id as ENCOUNTER','ORGANIZATION', 'PROVIDER', 'PAYER')
b = a.toPandas().to_dict(orient='list')
b

B41D26FD-733E-7773-0115-A9E5C0451FBC
+------------------------------------+-------------------+-------------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+--------------+---------+------------------------------------------+-------------------+----------------+--------------+----------+-----------------+
|Id                                  |START              |STOP               |PATIENT                             |ORGANIZATION                        |PROVIDER                            |PAYER                               |ENCOUNTERCLASS|CODE     |DESCRIPTION                               |BASE_ENCOUNTER_COST|TOTAL_CLAIM_COST|PAYER_COVERAGE|REASONCODE|REASONDESCRIPTION|
+------------------------------------+-------------------+-------------------+------------------------------------+------------------------------------+------------------------------------+----------------------

{'ENCOUNTER': ['b41d26fd-733e-7773-0115-a9e5c0451fbc'],
 'ORGANIZATION': ['f2068cee-c75c-321d-9b2c-c33535db89c9'],
 'PROVIDER': ['56da1c7e-8611-3744-9fb6-4e9f43aee2f6'],
 'PAYER': ['d18ef2e6-ef40-324c-be54-34a5ee865625']}

In [8]:
print("================")
print("organizations.csv")
print("================")

from pyspark.sql.functions import col
c = b['ORGANIZATION']
SOURCE_DATASET_FILENAME = "organizations.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(col("Id").isin(c))

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

organizations.csv
+------------------------------------+------------------------------+------------+--------+-----+--------+-----------+-----------+----------+-------+-----------+
|Id                                  |NAME                          |ADDRESS     |CITY    |STATE|ZIP     |LAT        |LON        |PHONE     |REVENUE|UTILIZATION|
+------------------------------------+------------------------------+------------+--------+-----+--------+-----------+-----------+----------+-------+-----------+
|f2068cee-c75c-321d-9b2c-c33535db89c9|CARING HEALTHCARE SERVICES LLC|32 BOSSI AVE|RANDOLPH|MA   |23682002|42.20017865|-71.0582074|6172300787|0.0    |100        |
+------------------------------------+------------------------------+------------+--------+-----+--------+-----------+-----------+----------+-------+-----------+

Record count : 1


In [9]:
print("================")
print("providers.csv")
print("================")

from pyspark.sql.functions import col
c = b['PROVIDER']
SOURCE_DATASET_FILENAME = "providers.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(col("Id").isin(c))

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

providers.csv
+------------------------------------+------------------------------------+----------------+------+----------------+------------+--------+-----+--------+-----------+-----------+----------+----------+
|Id                                  |ORGANIZATION                        |NAME            |GENDER|SPECIALITY      |ADDRESS     |CITY    |STATE|ZIP     |LAT        |LON        |ENCOUNTERS|PROCEDURES|
+------------------------------------+------------------------------------+----------------+------+----------------+------------+--------+-----+--------+-----------+-----------+----------+----------+
|56da1c7e-8611-3744-9fb6-4e9f43aee2f6|f2068cee-c75c-321d-9b2c-c33535db89c9|Octavio Prosacco|M     |GENERAL PRACTICE|32 BOSSI AVE|RANDOLPH|MA   |23682002|42.20017865|-71.0582074|100       |0         |
+------------------------------------+------------------------------------+----------------+------+----------------+------------+--------+-----+--------+-----------+-----------+---------

In [10]:
print("================")
print("payers.csv")
print("================")

from pyspark.sql.functions import col
c = b['PAYER']
SOURCE_DATASET_FILENAME = "payers.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(col("Id").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

payers.csv
+------------------------------------+-------------+----------+-------+----+-------------------+----+-----+--------------+----------------+-------+------------------+--------------------+-------------------+---------------------+------------------+--------------------+---------------------+-----------------------+----------------+------------------+-------------+
|Id                                  |NAME         |OWNERSHIP |ADDRESS|CITY|STATE_HEADQUARTERED|ZIP |PHONE|AMOUNT_COVERED|AMOUNT_UNCOVERED|REVENUE|COVERED_ENCOUNTERS|UNCOVERED_ENCOUNTERS|COVERED_MEDICATIONS|UNCOVERED_MEDICATIONS|COVERED_PROCEDURES|UNCOVERED_PROCEDURES|COVERED_IMMUNIZATIONS|UNCOVERED_IMMUNIZATIONS|UNIQUE_CUSTOMERS|QOLS_AVG          |MEMBER_MONTHS|
+------------------------------------+-------------+----------+-------+----+-------------------+----+-----+--------------+----------------+-------+------------------+--------------------+-------------------+---------------------+------------------+---------

In [11]:
print("================")
print("claims.csv")
print("================")

SOURCE_DATASET_FILENAME = "claims.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(f'upper(APPOINTMENTID) = "{ENCOUNTERID}"').orderBy('Id')

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

claims.csv
+------------------------------------+------------------------------------+------------------------------------+------------------------------------+---------------------------+------------+-------------------+----------+----------+----------+----------+----------+----------+----------+----------+-------------------+------------------------------------+-------------------+-------------------+------------------------------------+-------+-------+-------+------------+------------+------------+-------------------+-------------------+-------------------+----------------------+----------------------+
|Id                                  |PATIENTID                           |PROVIDERID                          |PRIMARYPATIENTINSURANCEID           |SECONDARYPATIENTINSURANCEID|DEPARTMENTID|PATIENTDEPARTMENTID|DIAGNOSIS1|DIAGNOSIS2|DIAGNOSIS3|DIAGNOSIS4|DIAGNOSIS5|DIAGNOSIS6|DIAGNOSIS7|DIAGNOSIS8|REFERRINGPROVIDERID|APPOINTMENTID                       |CURRENTILLNESSDATE |SERVICEDATE 

In [13]:
# Select the Encounter ID
CLAIMID = '076a15e9-1da5-ada1-699d-aef0b51e534b'
CLAIMID = CLAIMID.upper()
print(CLAIMID)

x = df.filter(f'upper(Id) = "{CLAIMID}"').orderBy('Id').selectExpr('Id as CLAIM')
y = x.toPandas().to_dict(orient='list')
b.update(y)
b

076A15E9-1DA5-ADA1-699D-AEF0B51E534B


{'ENCOUNTER': ['b41d26fd-733e-7773-0115-a9e5c0451fbc'],
 'ORGANIZATION': ['f2068cee-c75c-321d-9b2c-c33535db89c9'],
 'PROVIDER': ['56da1c7e-8611-3744-9fb6-4e9f43aee2f6'],
 'PAYER': ['d18ef2e6-ef40-324c-be54-34a5ee865625'],
 'CLAIM': ['076a15e9-1da5-ada1-699d-aef0b51e534b']}

In [14]:
print("================")
print("claims_transactions.csv")
print("================")

CLAIMID = b['CLAIM'][0]
SOURCE_DATASET_FILENAME = "claims_transactions.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(f'upper(CLAIMID) = upper("{CLAIMID}")').orderBy('CLAIMID', 'CHARGEID')

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

x = df.filter(f'upper(CLAIMID) = upper("{CLAIMID}")').orderBy('CLAIMID', 'CHARGEID').selectExpr('ID as CLAIM_TRANSACTION')
y = x.toPandas().to_dict(orient='list')
b.update(y)
b


claims_transactions.csv


+------------------------------------+------------------------------------+--------+------------------------------------+-------+------+------+-------------------+-------------------+------------------------------------+---------------+---------+---------+-------------+-------------+-------------+-------------+-----+------------+------------------------------------------------------------------------------------+----------+-------------+------------+--------+-----------+---------+-----------+------------------------------------+--------+------------------------------------+-------------+------------------------------------+------------------------------------+
|ID                                  |CLAIMID                             |CHARGEID|PATIENTID                           |TYPE   |AMOUNT|METHOD|FROMDATE           |TODATE             |PLACEOFSERVICE                      |PROCEDURECODE  |MODIFIER1|MODIFIER2|DIAGNOSISREF1|DIAGNOSISREF2|DIAGNOSISREF3|DIAGNOSISREF4|UNITS|DEPARTMENTID|

{'ENCOUNTER': ['b41d26fd-733e-7773-0115-a9e5c0451fbc'],
 'ORGANIZATION': ['f2068cee-c75c-321d-9b2c-c33535db89c9'],
 'PROVIDER': ['56da1c7e-8611-3744-9fb6-4e9f43aee2f6'],
 'PAYER': ['d18ef2e6-ef40-324c-be54-34a5ee865625'],
 'CLAIM': ['076a15e9-1da5-ada1-699d-aef0b51e534b'],
 'CLAIM_TRANSACTION': ['64144591-798b-8f8f-979f-64fbacbbd8d9',
  '38288b22-f317-1f1f-5f3e-c9f75977b1b3',
  '66a22c8b-cc5c-7c7e-02fb-27dd65dec6cf',
  'cd445917-98b8-f8ff-05f6-4fbacbbd8da1',
  '3511645e-62e3-e409-17d9-3eeb2ef63696',
  '1798b8f9-2ffd-4459-bacb-bd8dd135f64f',
  '5e62e3e4-c9f5-1164-eb2e-f63756d7d93e',
  '798b8f93-63d4-4591-acbb-d8dd975f64fb',
  'e62e3e4e-af51-1645-b2ef-63777d7d93ee',
  '98b8f93f-d544-5917-cbbd-8de10df64fba',
  'b682a22c-8bcc-5c7c-071e-fb27dd65dec7']}

In [15]:
print("================")
print("allergies.csv")
print("================")
SOURCE_DATASET_FILENAME = "allergies.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

allergies.csv
+-----+----+-------+---------+----+------+-----------+----+--------+---------+------------+---------+---------+------------+---------+
|START|STOP|PATIENT|ENCOUNTER|CODE|SYSTEM|DESCRIPTION|TYPE|CATEGORY|REACTION1|DESCRIPTION1|SEVERITY1|REACTION2|DESCRIPTION2|SEVERITY2|
+-----+----+-------+---------+----+------+-----------+----+--------+---------+------------+---------+---------+------------+---------+
+-----+----+-------+---------+----+------+-----------+----+--------+---------+------------+---------+---------+------------+---------+

Record count : 0


In [16]:
print("================")
print("careplans.csv")
print("================")
SOURCE_DATASET_FILENAME = "careplans.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

careplans.csv
+------------------------------------+----------+----+------------------------------------+------------------------------------+---------+------------------------------------------+----------+---------------------------------+
|Id                                  |START     |STOP|PATIENT                             |ENCOUNTER                           |CODE     |DESCRIPTION                               |REASONCODE|REASONDESCRIPTION                |
+------------------------------------+----------+----+------------------------------------+------------------------------------+---------+------------------------------------------+----------+---------------------------------+
|3ebc9735-d1f3-d374-863d-f64eb709f04f|2019-03-19|NULL|751e1f62-93c3-8048-7f77-48415a978963|b41d26fd-733e-7773-0115-a9e5c0451fbc|443402002|Lifestyle education regarding hypertension|59621000  |Essential hypertension (disorder)|
|4c67fae7-ed21-7335-7353-ea6260b2e6a8|2019-03-19|NULL|751e1f62-93c3-8048-7f77-

In [17]:
print("================")
print("conditions.csv")
print("================")
SOURCE_DATASET_FILENAME = "conditions.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

conditions.csv
+----------+----------+------------------------------------+------------------------------------+---------+---------------------------------+
|START     |STOP      |PATIENT                             |ENCOUNTER                           |CODE     |DESCRIPTION                      |
+----------+----------+------------------------------------+------------------------------------+---------+---------------------------------+
|2019-03-19|NULL      |751e1f62-93c3-8048-7f77-48415a978963|b41d26fd-733e-7773-0115-a9e5c0451fbc|59621000 |Essential hypertension (disorder)|
|2019-03-19|2020-03-24|751e1f62-93c3-8048-7f77-48415a978963|b41d26fd-733e-7773-0115-a9e5c0451fbc|314529007|Medication review due (situation)|
|2019-03-19|NULL      |751e1f62-93c3-8048-7f77-48415a978963|b41d26fd-733e-7773-0115-a9e5c0451fbc|15777000 |Prediabetes                      |
|2019-03-19|NULL      |751e1f62-93c3-8048-7f77-48415a978963|b41d26fd-733e-7773-0115-a9e5c0451fbc|271737000|Anemia (disorder)         

In [18]:
print("================")
print("devices.csv")
print("================")

SOURCE_DATASET_FILENAME = "devices.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

devices.csv
+-------------------+----+------------------------------------+------------------------------------+---------+-------------------------------------+------------------------------------------------------------------------+
|START              |STOP|PATIENT                             |ENCOUNTER                           |CODE     |DESCRIPTION                          |UDI                                                                     |
+-------------------+----+------------------------------------+------------------------------------+---------+-------------------------------------+------------------------------------------------------------------------+
|2019-03-19 15:42:22|NULL|751e1f62-93c3-8048-7f77-48415a978963|b41d26fd-733e-7773-0115-a9e5c0451fbc|337414009|Blood glucose meter (physical object)|(01)66338823500868(11)190226(17)440312(10)57489205003913368(21)827494046|
+-------------------+----+------------------------------------+------------------------------------+

In [19]:
print("================")
print("imaging_studies.csv")
print("================")

SOURCE_DATASET_FILENAME = "imaging_studies.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

imaging_studies.csv
+---+----+-------+---------+----------+-------------+--------------------+-------------+--------------------+------------+--------+---------------+--------------+
|Id |DATE|PATIENT|ENCOUNTER|SERIES_UID|BODYSITE_CODE|BODYSITE_DESCRIPTION|MODALITY_CODE|MODALITY_DESCRIPTION|INSTANCE_UID|SOP_CODE|SOP_DESCRIPTION|PROCEDURE_CODE|
+---+----+-------+---------+----------+-------------+--------------------+-------------+--------------------+------------+--------+---------------+--------------+
+---+----+-------+---------+----------+-------------+--------------------+-------------+--------------------+------------+--------+---------------+--------------+

Record count : 0


In [20]:
print("================")
print("immunizations.csv")
print("================")

SOURCE_DATASET_FILENAME = "immunizations.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

immunizations.csv
+-------------------+------------------------------------+------------------------------------+----+--------------------------------------------------+---------+
|DATE               |PATIENT                             |ENCOUNTER                           |CODE|DESCRIPTION                                       |BASE_COST|
+-------------------+------------------------------------+------------------------------------+----+--------------------------------------------------+---------+
|2019-03-19 15:42:22|751e1f62-93c3-8048-7f77-48415a978963|b41d26fd-733e-7773-0115-a9e5c0451fbc|140 |Influenza  seasonal  injectable  preservative free|136.0    |
+-------------------+------------------------------------+------------------------------------+----+--------------------------------------------------+---------+

Record count : 1


In [21]:
print("================")
print("medications.csv")
print("================")
SOURCE_DATASET_FILENAME = "medications.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

medications.csv
+-------------------+-------------------+------------------------------------+------------------------------------+------------------------------------+-------+---------------------------------------------------------------------------+---------+--------------+---------+---------+----------+---------------------------------+
|START              |STOP               |PATIENT                             |PAYER                               |ENCOUNTER                           |CODE   |DESCRIPTION                                                                |BASE_COST|PAYER_COVERAGE|DISPENSES|TOTALCOST|REASONCODE|REASONDESCRIPTION                |
+-------------------+-------------------+------------------------------------+------------------------------------+------------------------------------+-------+---------------------------------------------------------------------------+---------+--------------+---------+---------+----------+---------------------------------+
|20

In [22]:
print("================")
print("observations.csv")
print("================")
SOURCE_DATASET_FILENAME = "observations.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

observations.csv
+-------------------+------------------------------------+------------------------------------+--------------+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------+-------+-------+
|DATE               |PATIENT                             |ENCOUNTER                           |CATEGORY      |CODE   |DESCRIPTION                                                                                                                                                                               |VALUE                                                                       |UNITS  |TYPE   |
+-------------------+------------------------------------+------------------------------------+--------------+-------+-------------------------------------------------------------------

In [23]:
print("================")
print("procedures.csv")
print("================")
SOURCE_DATASET_FILENAME = "procedures.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

procedures.csv
+-------------------+-------------------+------------------------------------+------------------------------------+---------------+------------------------------------------------------------------------------------+---------+----------+-----------------+
|START              |STOP               |PATIENT                             |ENCOUNTER                           |CODE           |DESCRIPTION                                                                         |BASE_COST|REASONCODE|REASONDESCRIPTION|
+-------------------+-------------------+------------------------------------+------------------------------------+---------------+------------------------------------------------------------------------------------+---------+----------+-----------------+
|2019-03-19 15:42:22|2019-03-19 16:12:31|751e1f62-93c3-8048-7f77-48415a978963|b41d26fd-733e-7773-0115-a9e5c0451fbc|710824005      |Assessment of health and social care needs (procedure)                              |4

In [24]:
print("================")
print("supplies.csv")
print("================")
SOURCE_DATASET_FILENAME = "supplies.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

supplies.csv
+----------+------------------------------------+------------------------------------+----------+----------------------------------------------+--------+
|DATE      |PATIENT                             |ENCOUNTER                           |CODE      |DESCRIPTION                                   |QUANTITY|
+----------+------------------------------------+------------------------------------+----------+----------------------------------------------+--------+
|2019-03-19|751e1f62-93c3-8048-7f77-48415a978963|b41d26fd-733e-7773-0115-a9e5c0451fbc|337388004 |Blood glucose testing strips (physical object)|50      |
|2019-03-19|751e1f62-93c3-8048-7f77-48415a978963|b41d26fd-733e-7773-0115-a9e5c0451fbc|1137596000|Blood lancet (physical object)                |100     |
+----------+------------------------------------+------------------------------------+----------+----------------------------------------------+--------+

Record count : 2


In [35]:
print("================")
print("payer_transitions.csv")
print("================")
SOURCE_DATASET_FILENAME = "payer_transitions.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = [PATIENTID.lower()]
a = df.filter(col("PATIENT").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

payer_transitions.csv
+------------------------------------+--------+-------------------+-------------------+------------------------------------+---------------+--------------+----------+
|PATIENT                             |MEMBERID|START_DATE         |END_DATE           |PAYER                               |SECONDARY_PAYER|PLAN_OWNERSHIP|OWNER_NAME|
+------------------------------------+--------+-------------------+-------------------+------------------------------------+---------------+--------------+----------+
|495425e0-b16b-0baa-f926-0dfbe9870673|NULL    |2013-10-30 16:26:49|2014-10-29 16:26:49|e03e23c9-4df1-3eb6-a62d-f70f02301496|NULL           |NULL          |NULL      |
|495425e0-b16b-0baa-f926-0dfbe9870673|NULL    |2014-10-29 16:26:49|2015-10-28 16:26:49|e03e23c9-4df1-3eb6-a62d-f70f02301496|NULL           |NULL          |NULL      |
|495425e0-b16b-0baa-f926-0dfbe9870673|NULL    |2015-10-28 16:26:49|2016-11-02 16:26:49|e03e23c9-4df1-3eb6-a62d-f70f02301496|NULL           |NUL

In [25]:
b

{'ENCOUNTER': ['b41d26fd-733e-7773-0115-a9e5c0451fbc'],
 'ORGANIZATION': ['f2068cee-c75c-321d-9b2c-c33535db89c9'],
 'PROVIDER': ['56da1c7e-8611-3744-9fb6-4e9f43aee2f6'],
 'PAYER': ['d18ef2e6-ef40-324c-be54-34a5ee865625'],
 'CLAIM': ['076a15e9-1da5-ada1-699d-aef0b51e534b'],
 'CLAIM_TRANSACTION': ['64144591-798b-8f8f-979f-64fbacbbd8d9',
  '38288b22-f317-1f1f-5f3e-c9f75977b1b3',
  '66a22c8b-cc5c-7c7e-02fb-27dd65dec6cf',
  'cd445917-98b8-f8ff-05f6-4fbacbbd8da1',
  '3511645e-62e3-e409-17d9-3eeb2ef63696',
  '1798b8f9-2ffd-4459-bacb-bd8dd135f64f',
  '5e62e3e4-c9f5-1164-eb2e-f63756d7d93e',
  '798b8f93-63d4-4591-acbb-d8dd975f64fb',
  'e62e3e4e-af51-1645-b2ef-63777d7d93ee',
  '98b8f93f-d544-5917-cbbd-8de10df64fba',
  'b682a22c-8bcc-5c7c-071e-fb27dd65dec7']}